In [1]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 4.9 MB/s eta 0:00:00


In [2]:
import json
import random
from groq import Groq
import nltk
import re
from scipy.optimize import linear_sum_assignment
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize

# Ensure you have the necessary NLTK data files
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
def read_claims_from_json(file_path):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
        return data
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return []
    except json.JSONDecodeError:
        print(f"Error decoding JSON from file: {file_path}")
        return []

# Load your dataset from JSON files
claims_file_1 = '/content/Modified_true_claims.json'
claims_file_2 = '/content/last_fake_claims.json'

claims_data_1 = read_claims_from_json(claims_file_1)
claims_data_2 = read_claims_from_json(claims_file_2)

# Combine claims from both files
combined_claims = claims_data_1 + claims_data_2
random.shuffle(combined_claims)

In [6]:
import time

In [7]:
import re


# Initialize Groq API client
client = Groq(api_key="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")



def convert_to_json(llm_response):
    responseQuestions = []
    pattern = r'\[(.*?)\]'
    matches = re.findall(pattern, llm_response, re.DOTALL)

    if not matches:
        return responseQuestions  # Return an empty list if no matches are found

    json_str = matches[0]
    questions_llm = re.findall(r'\{(.*?)\}', json_str, re.DOTALL)

    for question in questions_llm:
        question_json = {}
        q = re.findall(r'.?question.?:\s*.?(.*),[\s\n]+.?questionType', question, re.DOTALL)
        if q:
            question_json['question'] = q[0].strip().strip('"')
        qt = re.findall(r'.?questionType.?:\s*.?(.*)[\'\"]', question, re.DOTALL)
        if qt:
            question_json['questionType'] = qt[0].strip().strip('"')
        responseQuestions.append(question_json)

    return responseQuestions


def get_response_llm(claim, questions, max_retries=3, retry_delay=2):
  prompt = f"{questions} \n Claim: {claim}. Provide me only with similar questions relevant for the claim given in similar json format. Provide boolean, Abstractive, Extractive questions."
  response = None
  for attempt in range(max_retries):
    try:
        response = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            model="gemma2-9b-it",
            max_tokens=4000,
        )
        response_content = response.choices[0].message.content.strip()
        if response_content:
            response_json = convert_to_json(response_content)
            return response_json
        else:
             print(f"Empty response on attempt {attempt + 1} for claim: '{claim}'")

    except Exception as e:
        if response and response.choices and response.choices[0].message:
            print(f"Error on attempt {attempt + 1} for claim '{claim}': {e}, {response.choices[0].message.content}")
        else:
            print(f"Error on attempt {attempt + 1} for claim '{claim}': {e}, Response not available")
        if attempt < max_retries - 1:
            time.sleep(retry_delay) # Wait before retrying
        continue # Try again
  print(f"Failed to get response after {max_retries} attempts for claim: '{claim}'")
  return [] # Return empty list if all retries fail

In [8]:
new_data = []

for data in combined_claims[:5008]:
  questions_prompt = []
  claim = data["claim"]
  for question in combined_claims[0]["questions"]:
    q = {"question": question["question"]}
    answer = question["answers"][0]
    #q['answer'] = answer['answer']
    q['questionType'] = answer['answer_type']
    questions_prompt.append(q)
  response = get_response_llm(claim, questions_prompt)
  data = {'claim': claim, 'generatedQuestions': response, 'goldQuestions': questions_prompt}
  new_data.append(data)

In [9]:
new_data

[{'claim': 'NIA files chargesheet against 2 for stealing hard disk from ins vikrant.',
  'generatedQuestions': [{'question': "Is the claim about the NIA filing a chargesheet true?'",
    'questionType': 'Boolean'},
   {'question': "What details are included in the chargesheet filed by the NIA?'",
    'questionType': 'Abstractive'},
   {'question': "Against whom did the NIA file the chargesheet?'",
    'questionType': 'Extractive'},
   {'question': "What type of item was stolen from INS Vikrant according to the NIA chargesheet?'",
    'questionType': 'Extractive'}],
  'goldQuestions': [{'question': 'Did the NIA file a chargesheet against two individuals for stealing a hard disk from INS Vikrant?',
    'questionType': 'Boolean'},
   {'question': 'What action did the NIA take in the case of the stolen hard disk from INS Vikrant?',
    'questionType': 'Abstractive'},
   {'question': 'What was the chargesheet filed by the NIA related to?',
    'questionType': 'Extractive'}]},
 {'claim': "Na

In [10]:
import numpy as np

In [11]:
hungarian_meteor_scores = []

def calculate_meteor_scores(generated_sequences, reference_sequences):
    """
    Calculate the METEOR scores between generated sequences and reference sequences.

    :param generated_sequences: List of generated sequences (list of strings)
    :param reference_sequences: List of reference sequences (list of strings)
    :return: A 2D list of METEOR scores where scores[i][j] is the score between generated_sequences[i] and reference_sequences[j]
    """
    scores = []
    for gen_seq in generated_sequences:
        row = []
        for ref_seq in reference_sequences:
            score = meteor_score([ref_seq], gen_seq)
            row.append(score)
        scores.append(row)
    return scores

def hungarian_meteor(generated_sequences, reference_sequences):
    """
    Apply the Hungarian Algorithm to find the optimal matching between generated sequences and reference sequences.

    :param generated_sequences: List of generated sequences (list of strings)
    :param reference_sequences: List of reference sequences (list of strings)
    :return: A list of tuples where each tuple contains the index of the generated sequence and the index of the reference sequence it is matched to
    """

    # Tokenize the sequences

    generated_sequences = [word_tokenize(seq) for seq in generated_sequences]
    reference_sequences = [word_tokenize(seq) for seq in reference_sequences]

    # Calculate the METEOR scores
    scores = calculate_meteor_scores(generated_sequences, reference_sequences)

    # Convert the scores to a cost matrix (negative scores because Hungarian Algorithm minimizes cost)
    cost_matrix = [[-score for score in row] for row in scores]

    # Apply the Hungarian Algorithm to find the optimal assignment
    try:
      row_ind, col_ind = linear_sum_assignment(cost_matrix)
    except ValueError as e:
      print(f"ravi,{generated_sequences},{reference_sequences},{scores}")
    # Calculate the total score using the Hungarian algorithm
    total_score = 0
    for i, j in zip(row_ind, col_ind):
        total_score += scores[i][j]

    # Normalize the total score by the number of reference sequences
    total_score = total_score / len(reference_sequences)

    # Best score for each generated question (max score across all references)
    best_scores = [max(row) for row in scores]

    return total_score, best_scores, list(zip(row_ind, col_ind))


for data in new_data:
  generated_sequences = [question['question'] for question in data['generatedQuestions'] if question.get("question")]
  reference_sequences = [question['question'] for question in data['goldQuestions']]
  if not generated_sequences or not reference_sequences:
      continue
  total_score, best_scores, optimal_matching = hungarian_meteor(generated_sequences, reference_sequences)

  # Print the best METEOR score for each generated question
  for i, score in enumerate(best_scores):
      print(f"Generated Question {i+1}: {''.join(generated_sequences[i])}")
      print(f"Best METEOR Score: {score:.4f}")

  # Print the total score (calculated from Hungarian matching)
  hungarian_meteor_scores.append(total_score)
  print(f"Total METEOR Score (normalized): {total_score:.4f}")

  # Optional: If you want to print the optimal matching from the Hungarian algorithm
  print("Optimal Matching from Hungarian Algorithm:")
  for i, j in optimal_matching:
      print(f"Generated Question {i+1} matched with Reference Question {j+1}")
  print("-" * 50)

# Calculate the average of all total METEOR scores
average_meteor_score = sum(hungarian_meteor_scores) / len(hungarian_meteor_scores) if hungarian_meteor_scores else 0
print(f"\nAverage METEOR Score: {average_meteor_score:.4f}")

Streaming output truncated to the last 5000 lines.
--------------------------------------------------
Generated Question 1: Did Priyanka Gandhi start wearing her grandmother’s saree at the beginning of the Uttar Pradesh election season?'
Best METEOR Score: 0.2033
Generated Question 2: What change in Priyanka Gandhi's attire coincided with the start of election season in Uttar Pradesh?'
Best METEOR Score: 0.1657
Generated Question 3: What type of garment did Priyanka Gandhi start wearing according to the claim?'
Best METEOR Score: 0.2710
Total METEOR Score (normalized): 0.1600
Optimal Matching from Hungarian Algorithm:
Generated Question 1 matched with Reference Question 1
Generated Question 2 matched with Reference Question 3
Generated Question 3 matched with Reference Question 2
--------------------------------------------------
Generated Question 1: Is the government providing free masks through a new program?'
Best METEOR Score: 0.0901
Generated Question 2: Describe the government's